In [1]:
def vietnam_incubation():
    import math
    import re
    import os
    import ast 
    import json
    import pandas as pd 
    import numpy as np
    import time
    from ibm_watson_machine_learning.foundation_models import Model
    from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
    from ibm_watson import DiscoveryV2
    from ibm_watson.discovery_v2 import DiscoveryV2, QueryLargePassages
    from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
    
    #========== watson discovery credentials========+
    
    DISCOVERY_USERNAME = "apikey"
    DISCOVERY_APIKEY = "xcKsSFr_8QRuo6Ahm7a5IMJIXgB4ypiuk099ta4wCKph"
    DISCOVERY_PROJECT = "5e9472d7-d723-4b25-afc4-a1a92b83ce88"
    DISCOVERY_COLLECTION ="91661ba7-cebc-ac5c-0000-018fec422801"
    DISCOVERY_URL = "https://api.us-south.discovery.watson.cloud.ibm.com/instances/a1633f77-4808-4aa0-8354-2fa91a6d9957"
    DISCOVERY_VERSION = '2019-11-29'
    authenticator = IAMAuthenticator(DISCOVERY_APIKEY)
    DISCOVERY= DiscoveryV2(version=DISCOVERY_VERSION, authenticator=authenticator)
    DISCOVERY.set_service_url(DISCOVERY_URL)
    
    #=========== watson discovery ====================+
    
    def query_data_only(question, max_per_document, collection_id):
        collection_list = [collection_id]
        user_question = question
        passages = QueryLargePassages(enabled = True, per_document=True, find_answers=True, characters = 900, max_per_document=max_per_document, max_answers_per_passage=20)

        query_result = DISCOVERY.query(
            project_id=DISCOVERY_PROJECT,
            # filter= f"document_id::{document_id}",
            collection_ids=collection_list,
            natural_language_query=user_question,
            passages=passages).get_result()

        combined_result = query_result
        return combined_result

    def regex_changer(query_result):
        words_to_remove = ['<em>', '</em>']
        pattern = '|'.join(re.escape(word) for word in words_to_remove)
        output_string = re.sub(pattern, '', query_result)
        return output_string


    def query_data_json(combined_result):
        json_file_list = []  # Create a list to store dictionaries

        for i in range(len(combined_result['results'])):
            doc_id = combined_result['results'][i]['document_id']
            text = ''.join(combined_result['results'][i]['text'])
            passage_text = []
            confidence_score = 0  # Initialize confidence_score


            for j in range(len(combined_result['results'][i]['document_passages'])):
                max_j = len(combined_result['results'][i]['document_passages'])
                start_offset = math.floor(combined_result['results'][i]['document_passages'][j]['start_offset'] / 1000) * 1000
                end_offset = math.ceil(combined_result['results'][i]['document_passages'][j]['end_offset'] / 1000) * 1008
                #bisa 1068
                result = text[start_offset:end_offset]
                combined_result_regex = regex_changer(result)
                passage_text.append(combined_result_regex)

                for k in range(len(combined_result['results'][i]['document_passages'][j]['answers'])):
                    max_k = len(combined_result['results'][i]['document_passages'][j]['answers'])
                    if max_j < 2 and k < len(combined_result['results'][i]['document_passages'][j]['answers']):
                        # Correct indentation for confidence_score calculation
                        confidence_score += combined_result['results'][i]['document_passages'][j]['answers'][k]['confidence']
                        final_confidence_score = confidence_score / max_k

                    elif max_j >=2 and k < len(combined_result['results'][i]['document_passages'][j]['answers']):
                        if k < len(combined_result['results'][i]['document_passages'][j]['answers']):
                            # print(f"cf1:{combined_result['results'][i]['document_passages'][j]['answers'][k]['confidence']}")
                            confidence_score += combined_result['results'][i]['document_passages'][j]['answers'][k]['confidence']
                        final_confidence_score = confidence_score/(max_j*max_k)
                    else:
                        print("should not used")

            if max_j < 2:
                final_confidence_score = confidence_score / len(combined_result['results'][i]['document_passages'][j]['answers'])
            elif max_j>=2:
                final_confidence_score = final_confidence_score
            else:
                pass

            json_file = {'doc_id': doc_id,
                        'passage_text': passage_text,
                        'confidence_score':final_confidence_score}


            json_file_list.append(json_file) 

        return json_file_list
    
    #=========== watsonx.ai===========+
    class WatsonXModel:
        api_key = 'ZGBpAS7eDCkYb2H1IYdFB1S6xm-VCBWkXW_fiAfWQZLC'
        project_id = "5d038e3b-fbdd-4844-9985-7020d7fa88c7"
        ibm_cloud_url = "https://us-south.ml.cloud.ibm.com"

        if api_key is None or ibm_cloud_url is None or project_id is None:
            raise Exception("Ensure you copied the .env file that you created earlier into the same directory as this notebook")
        else:
            creds = {
                "url": ibm_cloud_url,
                "apikey": api_key
            }
        def __init__(
            self,
            creds=creds,
            project_id=project_id,
            model_name="meta-llama/llama-3-70b-instruct",
            model_params = {
                GenParams.DECODING_METHOD: "greedy",
                GenParams.MIN_NEW_TOKENS: 1,
                GenParams.MAX_NEW_TOKENS: 2000,
                GenParams.TEMPERATURE: 0.75,
                GenParams.REPETITION_PENALTY: 1,
                GenParams.TOP_K: 1,
                GenParams.RANDOM_SEED: 362,
                GenParams.STOP_SEQUENCES: ["[/INST]", " # ", "<|endoftext|>", "========================"],
            }
        ):
            model = Model(
                model_id=model_name,
                params=model_params,
                credentials=creds,
                project_id=project_id
            )

            self.model = model
        def qa_result(self, question, paragraph_text, conversation_history):
            model = self.model
            input_prompts = f"""
            <|begin of text|>
             <|start_header_id|>system<|end_header_id|>
             You are an AI gene model for the Q&A task, you will be given a 'question' and answer the question only based on the 'text_paragraph' given.
             Apart from that, there will be a 'conversation_history' which functions to provide context for previous conversations.
             If you feel the new question is relevant to the previous question, link the answer. If not, just answer the question without relating it to 'conversation history'.

             Paragraph_Text:
             {paragraph_text}

             conversation_history:
             {conversation_history}

             Answer the questions given by simply providing the answer, make sure the answer is detailed and comprehensive and extract the information you are trying to find from the conversation.
             Write the answer in the following JSON format:
             {{"result":{{"question":Asked question, "answer":"answer to question"}}}}
             Make sure to only provide JSON answers.
             <|eot_id|>

             <|start_header_id|>example<|end_header_id|>
             Question:
             What is New Renewable Energy?
             Answer:
             {{"result":{{"question":"What is new renewable energy?","answer":"New and Renewable Energy (EBT) is a topic that is increasingly attracting attention throughout the world for the environmental and energy sectors. In the middle concerns about climate change and limited fossil resources, the potential for renewable energy in Indonesia has great opportunities"}}}}

             Question:
             {question}
             <|eot_id|>
             Answer:
            """
            generated_response = model.generate_text(prompt=input_prompts)
            generated_response = generated_response.strip()
            return generated_response

        def relevancy_check_and_keyword(self, conversation_history, new_question):
            model = self.model
            input_prompts =f"""
           <|begin of text|>
             <|start_header_id|>system<|end_header_id|>
             You are an AI gene model whose job is to see whether "conversation_history" is related to "new_question" and make changes to the question sentence asked
             Here are some things that might happen:
             1. If "new_question" uses the word reference, then "new_question" is relevant to "convertsation_history" so is_relevant is yes.
             2. If the subject of "new_question" is the same as "conversation_history", then it is still relevant so is_relevant is yes.
             3. If "new_question" is like following up on "conversation_history" fill in is_relevant with yes
             4. If "new_question" does not contain a subject, then "new_question" is considered relevant to "conversation_history" so is_relevant is no.
             5. If "new_question" asks something different from "conversation_history", then it is considered irrelevant so is_relevant is no.
             6. If "conversation_history" is null and "new_question" has a value, then it is considered irrelevant is_relevant is no.
             Give a response between yes or no.

             If is_relevant is "yes" then change "new_question" by extracting information from "convertsation_history question section", "conversation_history answer section" and "new_question"
             into a coherent question sentence and put it in the "updated_question" variable

             If is_relevant is "no" the contents of "updated_question" remain the same as "new_question"
             <|eot_id|>

             <|start_header_id|>example<|end_header_id|>
             Example 1:
             new_question : What is the company's financial status?
             conversation_history: f"{{\"result\":{{"question":"Explain everything you know about Pertamina",\"answer\":"Pertamina is a state-owned company operating in the energy sector and the largest dividend contributor in Indonesia. Having an integrated upstream to downstream business, starting from Exploration and Production, Processing, Distribution and Marketing, which is based on AKHLAK values ​​as core values."}}}}"
             response: f"{{{{\"result\":{{\"is_relevant\":\"yes\",\"updated_question\":\"What is Pertamina's financial status?\"}}}}}}"

             Example 2:
             new_question: Can you explain in more detail?
             conversation_history: f"{{\"result\":{{"question":"How much will Pertamina profit in 2022?",\"answer\":"Pertamina's profit achievement also made history, reaching IDR 60 trillion. He said, this also cannot be separated from the company's efforts to continue to make various efficiencies."}}}}"
             response: f"{{{{\"result\":{{\"is_relevant\":\"yes\",\"updated_question\":\"Can you explain further regarding Pertamina's profit in 2022\"}} }}}}"

             Example 3:
             new_question: What do you know about 3 Kg LPG gas?
             convertsation_history: f"{{\"result\":{{"question":"What year was Pertamina founded?",\"answer\":"Pertamina was founded on December 10, 1957"}}}}"
             response: f"{{{{\"result\":{{\"is_relevant\":\"no\",\"updated_question\":\"What do you know about 3 Kg LPG gas?\"}} }}}}"

             Example 4:
             new_question: What do you know about 3 Kg LPG gas?
             conversation_history: null
             response: f"{{{{\"result\":{{\"is_relevant\":\"no\",\"updated_question\":\"What do you know about 3 Kg LPG gas?\"}} }}}}"

             Example 5:
             new_question: How many subholdings does Pertamina have?
             convertsation_history: f"{{\"result\":{{"question":"What is called OVOO?",\"answer\":"One Village One Outlet (OVOO) to expand the distribution of Liquified Petroleum Gas (LPG) three kilograms (kg) in North Sulawesi (Sulut) Province. "Currently in North Sulawesi there are 4,052 3 kg LPG bases spread across 127 sub-districts or 1,408 sub-districts"}}}}"
             response: f"{{{{\"result\":{{\"is_relevant\":\"no\",\"updated_question\":\"How many subholdings is Pertamina?\"}}}}}}"

             Example 6:
             new_question: What is the positive impact of this program
             convertsation_history: f"{{\"result\":{{"question":"What is the One Price Fuel Program?",\"answer\":"Program to equalize fuel prices throughout Indonesia"}}}}"
             response: f"{{{{\"result\":{{\"is_relevant\":\"yes\",\"updated_question\":\"What is the positive impact of the One Price BBM program?\"}}}}}}"
             <|eot_id|>

             <|start_header_id|>assistant<|end_header_id|>
             Only provide responses in valid JSON format!

             new_question: {new_question}
             previous_question: {conversation_history}
             response:
             <|eot_id|>
            """

            generated_response = model.generate_text(prompt=input_prompts)
            generated_response = generated_response.strip()
            return generated_response

        
    
    def question_answer_flow(question, conversation_history):
        wx = WatsonXModel()
        try: 
            previous_question = conversation_history['result']['question']
        except:
            previous_question = ""
        relevan_result = wx.relevancy_check_and_keyword(previous_question, question)
        relevan_result = ast.literal_eval(relevan_result)
        #display(relevan_result)
        status_relevancy = relevan_result['result']['is_relevant']
        if status_relevancy == "no":
            conversation_history = ""
            wd_result  = query_data_only(question=question, max_per_document=4, collection_id=DISCOVERY_COLLECTION)
            wd_result = query_data_json(wd_result)
            qa_result = wx.qa_result(question=question, paragraph_text =wd_result[0]['passage_text'], conversation_history=conversation_history)
            qa_result = ast.literal_eval(qa_result)
        else:
            new_extracted_question = relevan_result['result']['updated_question']
            wd_result  = query_data_only(question=new_extracted_question, max_per_document=4, collection_id=DISCOVERY_COLLECTION)
            wd_result = query_data_json(wd_result)
            qa_result = wx.qa_result(question=new_extracted_question, paragraph_text =wd_result[0]['passage_text'], conversation_history=conversation_history)
            qa_result = ast.literal_eval(qa_result)
        conversation_history = qa_result
        return qa_result, conversation_history, status_relevancy, previous_question, wd_result
    
    #================= Main Functions ================+
    def vietnam_incubation(payload):
        start_time = time.time()
        json_qna = {}
        for data in payload['input_data']:
            values = data['values'][0][0]
            question = values['question']
            conversation_history = values['conversation_history']
            qa_result, conversation_history_final, status_relevancy, prev_question, wd_result = question_answer_flow(question, conversation_history)
            json_qna['qa_result'] = qa_result
            json_qna['conversation_history'] = conversation_history_final
            json_qna['status_relevancy'] = status_relevancy
            json_qna['previous_question'] = prev_question
            json_payload_result = []
            json_payload_result.append(json_qna)
            json_payload_result
        end_time = time.time()
        delta_time_sec = end_time - start_time
        return {'predictions': [{'fields': ['result'], 'values': [{'process_result':json_payload_result, 'eta':delta_time_sec }]}]}
    return vietnam_incubation

In [16]:
question = "Where is the experiment is carried out?"
conversation_history = {'result': {"question" : "How much water absorb?",
                                    'answer': 'It is absorb 97% water to transport system'}}
# conversation_history = ""
payload = {
    "input_data": [{
        "fields": ["query"],
        "values": [[{"question":question, "conversation_history":conversation_history}]]
}]}
vietnam_incubation()(payload)

{'result': {'is_relevant': 'no',
  'updated_question': 'Where is the experiment is carried out?'}}

{'predictions': [{'fields': ['result'],
   'values': [{'process_result': [{'qa_result': {'result': {'question': 'Where is the experiment carried out?',
         'answer': 'The experiment was carried out in a greenhouse at the Federal University of São João del-Rei (UFSJ), in the municipality of Sete Lagoas, Minas Gerais State, Brazil (19° 28’ 32” S, 44° 11’ 44” W).'}},
       'conversation_history': {'result': {'question': 'Where is the experiment carried out?',
         'answer': 'The experiment was carried out in a greenhouse at the Federal University of São João del-Rei (UFSJ), in the municipality of Sete Lagoas, Minas Gerais State, Brazil (19° 28’ 32” S, 44° 11’ 44” W).'}},
       'status_relevancy': 'no',
       'previous_question': 'How much water absorb?'}],
     'eta': 8.622572660446167}]}]}

In [2]:
api_key = 'ZGBpAS7eDCkYb2H1IYdFB1S6xm-VCBWkXW_fiAfWQZLC'
project_id = "1eb14bcf-6e24-47ec-a8a4-7748def62568"
ibm_cloud_url = "https://us-south.ml.cloud.ibm.com"
space_id = "7ee16482-8596-4c98-9a45-0f96e3eeec15"
location = "us-south"
wml_credentials = {
    "apikey": api_key,
    "url": 'https://' + location + '.ml.cloud.ibm.com'
}
from ibm_watson_machine_learning import APIClient
import requests
import json

client = APIClient(wml_credentials)

In [3]:
#if you want to redeploy just re run this model
client.set.default_space(space_id)
sofware_spec_uid = client.software_specifications.get_id_by_name("incu-vietnam")
meta_data = {
    client.repository.FunctionMetaNames.NAME:'Incubation-Vietnam-V3',
    client.repository.FunctionMetaNames.SOFTWARE_SPEC_UID: sofware_spec_uid
}
function_details = client.repository.store_function(meta_props=meta_data, function=vietnam_incubation)

In [20]:
function_uid = client.repository.get_function_uid(function_details)
# Deploy the stored function

metadata = {
    client.deployments.ConfigurationMetaNames.NAME: "Vietnam-Incubation",
    client.deployments.ConfigurationMetaNames.ONLINE: {}
}
function_deployment_details = client.deployments.create(function_uid, meta_props=metadata)



#######################################################################################

Synchronous deployment creation for uid: 'f7e4e78d-e783-4824-9b56-7b9dcaef79dc' started

#######################################################################################


initializing
Note: online_url and serving_urls are deprecated and will be removed in a future release. Use inference instead.
....................................
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='226dc1b6-54a4-4f71-b35b-f436158d1bc2'
------------------------------------------------------------------------------------------------




In [21]:
# Get the endpoint URL of the function deployment just created
function_deployment_id = client.deployments.get_uid(function_deployment_details)
function_deployment_endpoint_url = client.deployments.get_scoring_href(function_deployment_details)
print("Function deployment id: {}".format(function_deployment_id))
print("Endpoint URL: {}".format(function_deployment_endpoint_url))

Function deployment id: 226dc1b6-54a4-4f71-b35b-f436158d1bc2
Endpoint URL: https://us-south.ml.cloud.ibm.com/ml/v4/deployments/226dc1b6-54a4-4f71-b35b-f436158d1bc2/predictions
